In [1]:
import os
import pickle

import pandas as pd

from tqdm.notebook import tqdm

from wbapi import WbApi

%load_ext autoreload
%autoreload 2

In [2]:
with open('token.txt') as f:
    token = f.read()

In [3]:
with WbApi.create_wb_api(token) as wb_api:
    print(wb_api.get_cards_by_vendors('PA1-79046|||CF 0301///1'))

[{'imtID': 114925413, 'nmID': 133782326, 'vendorCode': 'PA1-79046|||CF 0301///1', 'mediaFiles': [], 'sizes': [{'techSize': '0', 'skus': ['PA1-79046|||CF 0301///1'], 'chrtID': 229583363, 'wbSize': '', 'price': 9049}], 'characteristics': [{'Ширина упаковки': 21}, {'Высота упаковки': 12}, {'Длина упаковки': 8}, {'Артикул производителя': ['CF 0301']}, {'Бренд': 'TRIALLI'}, {'Страна производства': ['Китай']}, {'Ставка НДС': ['20']}, {'Комплектация': ['1']}, {'Предмет': 'Цилиндры автомобильные'}, {'Наименование': 'Цилиндр главный тормозной Газель Бизнес дв УМЗ-4216 CF0301'}, {'Описание': 'Цилиндр главный тормозной Газель Бизнес  дв УМЗ-4216 CF0301 Gazelle-Business          '}], 'isProhibited': False}]


In [4]:
vendors = pd.read_excel('data\\vendors_add_nds.xlsx')

In [18]:
problem_index = []

In [19]:
with WbApi.create_wb_api(token) as wb_api:
    start = 0
    for i in tqdm(
        range(start * 100, vendors.shape[0], 100),
        initial=start,
        total=vendors.shape[0] // 100 + 1
    ):
        vendor_frame = vendors['Артикул продавца'].values[i: i + 100].tolist()
        cards = wb_api.get_cards_by_vendors(vendor_frame)
        result = wb_api.add_nds(cards)
        if not result:
            print(f'Problem in index {i}')
            problem_index.append(i)

  0%|          | 0/6522 [00:00<?, ?it/s]

<Response [500]>
{'data': None, 'error': True, 'errorText': 'Внутренняя ошибка', 'additionalErrors': {'uploadMany.CreateCardsToProcess': 'failed to insert source cards: timeout: context deadline exceeded'}}
Problem in index 316700
Ошибка. Код ответа - 504. Повторный запрос...
<Response [400]>
{'data': None, 'error': True, 'errorText': 'Неправильный формат запроса, кол-во изменяемых карточек товаров не должно быть 0', 'additionalErrors': {'Неправильный формат запроса, кол-во изменяемых карточек товаров не должно быть 0': 'None'}}
Problem in index 464700
<Response [400]>
{'data': None, 'error': True, 'errorText': 'Неправильный формат запроса, кол-во изменяемых карточек товаров не должно быть 0', 'additionalErrors': {'Неправильный формат запроса, кол-во изменяемых карточек товаров не должно быть 0': 'None'}}
Problem in index 464800
<Response [400]>
{'data': None, 'error': True, 'errorText': 'Неправильный формат запроса, кол-во изменяемых карточек товаров не должно быть 0', 'additionalErro

In [22]:
problem_index

[316700, 464700, 464800, 464900, 465000, 465100, 465200]

In [24]:
with WbApi.create_wb_api(token) as wb_api:
    while problem_index:
        i = problem_index.pop()
        vendor_frame = vendors['Артикул продавца'].values[i: i + 100].tolist()
        cards = wb_api.get_cards_by_vendors(vendor_frame)
        result = wb_api.add_nds(cards)
        if not result:
            print(f'Problem in index {i}')
            problem_index.append(i)
        print(len(problem_index))
    

KeyboardInterrupt: 